In [1]:
import pandas as pd
from binance.um_futures import UMFutures
from datetime import datetime, timedelta
import pytz

def get_binance_data(symbol="BTCUSDT", interval="5m", date="2025-06-01", duration_hours=24):
    """
    Get Binance futures kline data for a specific date and duration
    
    Args:
        symbol: Trading pair (default: "BTCUSDT")
        interval: Kline interval (default: "5m")
        date: Date in format "YYYY-MM-DD" (default: "2025-06-01")
        duration_hours: Duration in hours (default: 24 for full day)
    
    Returns:
        DataFrame with OHLCV data
    """
    um_futures_client = UMFutures()
    
    # Convert date string to datetime in UTC
    start_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = start_date.replace(tzinfo=pytz.UTC)  # Set timezone to UTC
    end_date = start_date + timedelta(hours=duration_hours)
    
    # Convert to milliseconds for Binance API
    start_time = int(start_date.timestamp() * 1000)
    end_time = int(end_date.timestamp() * 1000)
    
    # Get data from Binance
    data = um_futures_client.klines(
        symbol=symbol, 
        interval=interval, 
        startTime=start_time,
        endTime=end_time,
        limit=1000
    )
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=[
        "datetime", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
        "ignore"
    ])
    
    return df

# Get 5-minute data for June 1, 2025 (full day) starting from UTC 00:00:00
data = get_binance_data(interval="15m", date="2025-06-03", duration_hours=24)
df = data

# Convert the 'datetime' column to a datetime object
df["datetime"] = pd.to_datetime(df["datetime"], unit="ms")

# Keep only the relevant columns
df = df[["datetime", "open", "high", "low", "close", "volume"]]
df = df.set_index("datetime")

# save the DataFrame to a CSV file
df.to_csv("BTCUSDT_2025-06-04_15min.csv")

In [7]:
# df_from_csv = pd.read_csv("BTCUSDT_2025-06-01_5min.csv", parse_dates=True)
# df_from_csv

val=(24*60)/5
val

288.0

### Getting daily data including previously 21 candle

In [17]:
import pandas as pd
from binance.um_futures import UMFutures
from datetime import datetime, timedelta
import pytz


def get_binance_data(symbol="BTCUSDT", interval="15m", date="2025-06-03", duration_hours=24, lookback_candles=21):
    """
    Fetch Binance futures kline data based on UTC day boundaries, including lookback candles.

    Args:
        symbol: Trading pair (e.g., "BTCUSDT")
        interval: Kline interval (e.g., "15m")
        date: Target day in "YYYY-MM-DD" (UTC)
        duration_hours: How many hours of data to fetch from the date (default: 24)
        lookback_candles: Number of candles from the *previous UTC day* to include

    Returns:
        pd.DataFrame with OHLCV data from (previous 21 candles + full UTC day)
    """
    um_futures_client = UMFutures()

    # Map interval to minutes
    interval_map = {
        "1m": 1, "3m": 3, "5m": 5, "15m": 15, "30m": 30,
        "1h": 60, "2h": 120, "4h": 240, "6h": 360,
        "8h": 480, "12h": 720, "1d": 1440
    }

    if interval not in interval_map:
        raise ValueError(f"Unsupported interval '{interval}'")

    candle_minutes = interval_map[interval]
    lookback_minutes = lookback_candles * candle_minutes

    # Parse the given date in UTC (00:00:00)
    utc = pytz.UTC
    start_utc = utc.localize(datetime.strptime(
        date, "%Y-%m-%d"))  # 2025-06-03 00:00:00 UTC
    # 2025-06-04 00:00:00 UTC
    end_utc = start_utc + timedelta(hours=duration_hours)
    lookback_start_utc = start_utc - \
        timedelta(minutes=lookback_minutes)  # Go back 21 candles

    # Convert to ms timestamps
    start_time = int(lookback_start_utc.timestamp() * 1000)
    end_time = int(end_utc.timestamp() * 1000)

    # Fetch candles
    data = um_futures_client.klines(
        symbol=symbol,
        interval=interval,
        startTime=start_time,
        endTime=end_time,
        limit=1500
    )

    # Create DataFrame
    df = pd.DataFrame(data, columns=[
        "datetime", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
        "ignore"
    ])

    df["datetime"] = pd.to_datetime(df["datetime"], unit="ms", utc=True)
    df = df[["datetime", "open", "high", "low", "close", "volume"]]
    df.set_index("datetime", inplace=True)

    return df


# ✅ Example usage
df = get_binance_data(
    symbol="SOLUSDT",
    interval="15m",
    date="2025-06-02",  # Treated as UTC day
    duration_hours=24,
    lookback_candles=21
)
# Remove timezone info for CSV output (make datetime naive but still in UTC)
df.index = df.index.tz_convert(None)
df.to_csv("SOLUSDT_2025-06-02_UTC_with_lookback.csv")
print(df.head(25))

                         open      high       low     close     volume
datetime                                                              
2025-06-01 18:45:00  153.4400  154.6800  153.3700  154.6400  276521.66
2025-06-01 19:00:00  154.6400  155.3500  154.4300  154.6700  272717.22
2025-06-01 19:15:00  154.6700  155.2100  154.5900  155.1600   75344.11
2025-06-01 19:30:00  155.1600  155.5400  154.9700  155.5000  141482.46
2025-06-01 19:45:00  155.5100  155.8800  155.4800  155.8100   71995.02
2025-06-01 20:00:00  155.8000  155.9000  155.0800  155.3400  139792.78
2025-06-01 20:15:00  155.3300  155.8300  155.3000  155.6800   64872.54
2025-06-01 20:30:00  155.6800  156.1800  155.5600  156.1000   98172.44
2025-06-01 20:45:00  156.1100  156.1300  155.4100  155.6100   95349.64
2025-06-01 21:00:00  155.6000  156.4400  155.4900  156.3800   97013.10
2025-06-01 21:15:00  156.3800  156.6300  156.1600  156.2200   86860.03
2025-06-01 21:30:00  156.2200  157.2000  156.2100  156.5700  127189.12
2025-0